<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Calculate-albedo" data-toc-modified-id="Calculate-albedo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Calculate albedo</a></span></li>


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import ticker, cm
from utility import analysis_parameters as ap, util
import numpy as np
import dask
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [ ]:
region="TWP"

In [ ]:
chunk_dict = {"time":500, "lat":400, "lon":1440} # "grid_size":500000} 
sa_olr = xr.open_dataset(ap.get_file("SAM", "rlt"), chunks=chunk_dict).rlt
sa_swn = xr.open_dataset(ap.get_file("SAM", "rst"), chunks=chunk_dict).rstacc
ds = xr.open_dataset(ap.get_file("DATA", "rad"), chunks=chunk_dict)
da_olr = ds.adj_atmos_lw_up_all_toa_1h
da_swu = ds.adj_atmos_sw_up_all_toa_1h
da_swd = ds.adj_atmos_sw_down_all_toa_1h
ni_olr = xr.open_dataset(ap.get_file("NICAM", "rlut"), chunks=chunk_dict).rlut
ni_swu = xr.open_dataset(ap.get_file("NICAM", "rsut"), chunks=chunk_dict).rsut
ni_swd = xr.open_dataset(ap.get_file("NICAM", "rsdt"), chunks=chunk_dict).rsdt
um_olr = xr.open_dataset(ap.get_file("UM", "rlut"), chunks=chunk_dict).rlut
um_swu = xr.open_dataset(ap.get_file("UM", "rsut"), chunks=chunk_dict).rsut
um_swd = xr.open_dataset(ap.get_file("UM", "rsdt"), chunks=chunk_dict).rsdt
ge_olr = xr.open_dataset(ap.get_file("GEOSr0.25deg", "rlut"), chunks=chunk_dict).rlut
ge_swu = xr.open_dataset(ap.get_file("GEOSr0.25deg", "rsut"), chunks=chunk_dict).rsut
ge_swn = xr.open_dataset(ap.get_file("GEOSr0.25deg", "rst"), chunks=chunk_dict).rst
ge_swd = ge_swu+ge_swn
# sc_olr = xr.open_dataset(ap.get_file("SCREAMr0.25deg", "rlt"), chunks=chunk_dict).rlt
# sc_swn = xr.open_dataset(ap.get_file("SCREAMr0.25deg", "rst"), chunks=chunk_dict).rst
# sc_swd = xr.open_dataset(ap.get_file("SCREAMr0.25deg", "rsdt"), chunks=chunk_dict).rsdt
ge_swu

In [ ]:
# sa_swn = sa_swn.where(sa_swn.time.dt.hour<4).where(sa_swd>400)
# sa_swd = sa_swd.where(sa_swd.time.dt.hour<4).where(sa_swd>400)
# sa_swu = sa_swu.where(sa_swu.time.dt.hour<4).where(sa_swd>400)
# sa_swn[:40,4,4].values, sa_swd[:40,4,4].values, sa_swd[:40,4,4].values-sa_swn[:40,4,4].values

# um_swn = um_swd-um_swu

In [ ]:
# fig, ax = plt.subplots(2,1,figsize=(12,6))
# ge_swn.mean(axis=(1,2)).plot(label="SW net", ax=ax[0])
# ge_swu.mean(axis=(1,2)).plot(label="upwelling SW", ax=ax[0])
# ge_swd.mean(axis=(1,2)).plot(label="downwelling SW", ax=ax[0])
# (ge_swu/ge_swd).mean(axis=(1,2)).plot(ax=ax[1])
# ax[1].set_ylim([0,0.8])
# fig.suptitle("GEOS TWP")
# ax[0].legend()
# plt.savefig("../plots/geos_twp-swd-swn-swu.png", dpi=120)
# plt.show()

# Coarsen 

In [ ]:
n=40 # n=40 for 0.25 deg or n=10 for 1 deg
ni_olr = ni_olr.coarsen(lat=len(ni_olr.lat)//n, lon=len(ni_olr.lon)//n, boundary='trim').mean()
ni_swu = ni_swu.coarsen(lat=len(ni_swu.lat)//n, lon=len(ni_swu.lon)//n, boundary='trim').mean()
ni_swd = ni_swd.coarsen(lat=len(ni_swd.lat)//n, lon=len(ni_swd.lon)//n, boundary='trim').mean()
sa_olr = sa_olr.coarsen(lat=len(sa_olr.lat)//n, lon=len(sa_olr.lon)//n, boundary='trim').mean()
# sa_swu = sa_swu.coarsen(lat=len(sa_swu.lat)//n, lon=len(sa_swu.lon)//n, boundary='trim').mean()
# sa_swd = sa_swd.coarsen(lat=len(sa_swd.lat)//n, lon=len(sa_swd.lon)//n, boundary='trim').mean()
sa_swn = sa_swn.coarsen(lat=len(sa_swn.lat)//n, lon=len(sa_swn.lon)//n, boundary='trim').mean()
um_olr = um_olr.coarsen(latitude=len(um_olr.latitude)//n, longitude=len(um_olr.longitude)//n, boundary='trim').mean()
um_swu = um_swu.coarsen(latitude=len(um_swu.latitude)//n, longitude=len(um_swu.longitude)//n, boundary='trim').mean()
um_swd = um_swd.coarsen(latitude=len(um_swd.latitude)//n, longitude=len(um_swd.longitude)//n, boundary='trim').mean()


In [ ]:
sa_swd = ge_swd.interp(time=sa_swn.time, 
                       lat=sa_swn.lat, 
                       lon=sa_swn.lon, 
                       method="nearest",
                       kwargs={"fill_value": np.nan})
sa_swu = sa_swd - sa_swn

# Calculate albedo


$ alb = \frac{sw_{up}}{sw_{down}} $ 

In [ ]:
ni_alb = ni_swu/ni_swd.where(ni_swd>100)
sa_alb = sa_swu/sa_swd.where(sa_swd>100)
um_alb = um_swu/um_swd.where(um_swd>100)
da_alb = da_swu/da_swd.where(da_swd>10)
ge_alb = ge_swu/ge_swd.where(ge_swd>100)
# ni_alb = ni_swu/np.where(ni_swd>10, sni_wd, np.nan)
# ni_alb = ni_swu/np.where(ni_swd>10, sni_wd, np.nan)
# ni_alb


In [ ]:
um_swn = um_swd-um_swu
fig, ax = plt.subplots(2,1,figsize=(12,6))
ge_swn.mean(axis=(1,2)).plot(label="SW net", ax=ax[0])
ge_swu.mean(axis=(1,2)).plot(label="upwelling SW", ax=ax[0])
ge_swd.mean(axis=(1,2)).plot(label="downwelling SW", ax=ax[0])
# ge_swd.mean(axis=(1,2)).plot(label="GEOS downwelling SW", linestyle="--", ax=ax[0])
ge_alb = (ge_swu/ge_swd)
ge_alb.where(ge_alb.time.dt.hour<5).mean(axis=(1,2)).plot(ax=ax[1], label="alb")
axt=ax[1].twinx()
ge_olr.mean(axis=(1,2)).plot(ax=axt, color="C1", label="OLR")
ax[1].legend(loc="upper left")
axt.legend(loc="upper right")
ax[1].set_ylabel("albedo")
ax[1].set_ylim([0,0.8])
fig.suptitle("GEOS TWP")
ax[0].legend()
plt.savefig("../plots/geos_coarse_twp-swd-swn-swu.png", dpi=120)
plt.show()

In [ ]:
da_olr.shape, da_alb.shape, da_swu.shape, da_swd.shape

In [ ]:
# models=["CERES SYN1deg 1 hrly", "NICAM 1x1", "SAM 1x1 - using NICAM SWD", "UM 1x1", "GEOS 1x1"]
models=["CERES SYN1deg 1 hrly", "NICAM 0.25deg", "SAM 0.25deg - using NICAM SWD", "UM 0.25deg", "GEOS 0.25deg"]
fig = plt.figure(figsize=(28,18))
axes = [fig.add_subplot(2,3,i,box_aspect=1.1) for i in range(1,len(models)+1)]
i = 0
olrs=[da_olr, ni_olr, sa_olr, um_olr, ge_olr]
albs=[da_alb, ni_alb, sa_alb, um_alb, ge_alb]
for i in range(len(olrs)):
    olr = olrs[i]
    alb = albs[i]
    olr = olr.where((olr.time.dt.hour>=0)&(olr.time.dt.hour<=4))
    alb = alb.where((alb.time.dt.hour>=0)&(alb.time.dt.hour<=4))
    util.dennisplot("density",olr.values.flatten(), alb.values.flatten(), 
                ax=axes[i], model=models[i], region=region, cmap="gist_earth_r")
    axes[i].set_xlabel("OLR (W/m$^2$)")
    axes[i].set_ylabel("Albedo")
    axes[i].grid(True)
    axes[i].set_ylim([0,0.8])
    axes[i].set_title("{} {}\n01 Jan-1 Mar 2020 (10am-2pm LT)".format(models[i], region), fontsize=20)
    i+=1
plt.savefig("../plots/TWP/jhist_{}.png".format(region))
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
t = 400
ge_olr[t].plot.pcolormesh(vmin=80, vmax=300, cmap="viridis_r", ax=ax[0])
ge_alb[t].plot.pcolormesh(vmin=0.05, vmax=0.8, cmap="viridis", ax=ax[1], label="Mean alb={0:0.2g}".format(ge_alb[t].mean().values))
ax[0].annotate("Mean OLR={}".format(int(ge_olr[t].mean().values)), xy=(0.1,0.1), xycoords="axes fraction", backgroundcolor="w")
ax[1].annotate("Mean Alb={0:0.2g}".format((ge_alb[t].mean().values)), xy=(0.1,0.1), xycoords="axes fraction", backgroundcolor="w")
plt.savefig("../plots/TWP/GEOS_weird_albolr.png")
plt.show()

print(ge_olr[t].mean().values, ge_alb[t].mean().values)


In [ ]:
ge_iwp = xr.open_dataset(ap.get_file("GEOSr0.25deg", "clivi", "TWP")).clivi
ge_iwp

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
t = 400
ge_olr[t].plot.pcolormesh(vmin=80, vmax=300, cmap="viridis_r", ax=ax[0])
ge_alb[t].plot.pcolormesh(vmin=0.05, vmax=0.8, cmap="viridis", ax=ax[1], label="Mean alb={0:0.2g}".format(ge_alb[t].mean().values))
ax[0].annotate("Mean OLR={}".format(int(ge_olr[t].mean().values)), xy=(0.1,0.1), xycoords="axes fraction", backgroundcolor="w")
ax[1].annotate("Mean Alb={0:0.2g}".format((ge_alb[t].mean().values)), xy=(0.1,0.1), xycoords="axes fraction", backgroundcolor="w")
plt.savefig("../plots/TWP/GEOS_weird_albolr.png")
plt.show()